In [0]:
Create or replace MATERIALIZED VIEW bolt_finint_int.silver.fs_paymentmethods as

with global_paymentmethod_entities as (

-- Comment out the below once we switch to the raw catalogs
select tenant, record.paymentMethod.*, 'AMER' as region
from bolt_dcp_prod.bronze.raw_s2s_payment_method_entities

UNION all

select 
tenant, 
record.paymentMethod.*,
'LATAM' as region
from bolt_dcp_prod.any_latam_bronze.raw_s2s_payment_method_entities

UNION all 

select tenant, record.paymentMethod.*, 'EMEA' as region
from bolt_dcp_prod.any_emea_bronze.raw_s2s_payment_method_entities

UNION ALL

select tenant, record.paymentMethod.*, 'APAC' as region
from bolt_raw_prod.commerce.raw_any_apac_s2s_payment_method_entities

----------------------------------------------------------------------

-- Comment out below to switch ALL regions to the raw catalogs
-- UNION ALL

-- select tenant, record.paymentMethod.*, 'AMER' as region
-- from bolt_raw_prod.commerce.raw_any_amer_s2s_payment_method_entities

-- UNION all

-- select tenant, record.paymentMethod.*, 'LATAM' as region
-- from bolt_raw_prod.commerce.raw_any_latam_s2s_payment_method_entities

-- UNION all 

-- select tenant, record.paymentMethod.*, 'EMEA' as region
-- from bolt_raw_prod.commerce.raw_any_emea_s2s_payment_method_entities

)
select
tenant,
id as paymentMethodId,
realm,
userid,
replace(provider,'PAYMENT_PROVIDER_','') as payment_provider,
type as paymentMethod_type,
UPPER(countryCode) AS countryCode,
providerAgreementReference,

UPPER(cardDetails.cardProvider) as cardDetails_cardProvider,
UPPER(cardDetails.cardPaymentMethod) as cardDetails_cardPaymentMethod,
UPPER(cardDetails.cardIssuingBank) as cardDetails_cardIssuingBank,
UPPER(cardDetails.fundingSource) as cardDetails_fundingSource,
cardDetails.cardSummary as cardDetails_cardSummary,
date_format(from_unixtime(cardDetails.cardExpiry), 'yyyy-MM') as cardDetails_cardExpiry,

UPPER(geoLocation.countryIso) as geoLocation_countryIso,
UPPER(geoLocation.subdivisionIso) as geoLocation_subdivisionIso,
UPPER(geoLocation.city) as geoLocation_city,
geoLocation.postalCode as geoLocation_postalCode,
geoLocation.timeZone as geoLocation_timeZone,
geoLocation.isAnonymized as geoLocation_isAnonymized,

case 
  when currentState.pending is not null then 'PENDING'
  when currentState.valid is not null then 'VALID'
  when currentState.invalid is not null then 'INVALID'
  when currentState.expired is not null then 'EXPIRED'
  when currentState.canceled is not null then 'CANCELED'
  when currentState.deleted is not null then 'DELETED'
  when currentState.timedout is not null then 'TIMED_OUT'
  else 'CREATED'
end as paymentMethod_status,

legacyPaymentMethodId,
legacyExportingSource,
providerMerchantAccount,
providerUserReference,
created as record_created_date,
currentState.occurred as record_updated_date

from global_paymentmethod_entities
where id is not null